In [1]:
from arcgis.gis import GIS, Item
import pandas as pd

import sys
sys.path.append('..')

import geometry_monkeypatch
from reach_tools import *

In [2]:
url_reach_line = r'https://services5.arcgis.com/12oODIpfxlRR11MF/arcgis/rest/services/Reach_Line/FeatureServer/0'
url_reach_centroid = r'https://services5.arcgis.com/12oODIpfxlRR11MF/arcgis/rest/services/Reach_Centroid/FeatureServer/0'
url_reach_points = r'https://services5.arcgis.com/12oODIpfxlRR11MF/arcgis/rest/services/reach_points/FeatureServer/0'

user = 'joel_mccune'

lst_reach_id_or = ['10386', '1482', '1484', '3037', '1485', '1483', '10402', '10721', '10390', '10391', '3662', '11033', 
                '1486', '10910', '10447', '1487', '2726', '2725', '10825', '1488', '10385', '10940', '1489', '3038', 
                '1490', '1491', '3039', '3078', '10950', '3756', '2728', '3193', '10919', '10401', '11031', '11030', 
                '10453', '3757', '10450', '10279', '3758', '3195', '10431', '3205', '642', '1492', '1493', '1494', 
                '4574', '1495', '1496', '3485', '2713', '1497', '3699', '10454', '10811', '10449', '10448', '1498', 
                '1499', '1500', '11004', '10415', '10416', '1501', '1502', '10921', '11038', '1503', '3759', '10952', 
                '1504', '3031', '3073', '3074', '10684', '1505', '3075', '10686', '11052', '1506', '4344', '4345', 
                '4346', '1507', '3084', '10434', '1508', '5782', '10442', '10441', '1509', '10437', '11035', '5426', 
                '10458', '10459', '10393', '1510', '3197', '10947', '10945', '10418', '1511', '2721', '10443', 
                '10398', '10445', '10399', '10388', '1512', '3079', '1523', '1513', '3080', '3081', '3082', '3083', 
                '2120', '1514', '10818', '10389', '3760', '4296', '2871', '10439', '10440', '10446', '1516', '1515', 
                '2870', '2868', '4312', '1517', '11057', '2539', '3199', '3200', '3175', '3035', '1519', '10432', 
                '1518', '3034', '10433', '1520', '4298', '10438', '1521', '1522', '3858', '3169', '10400', '1524', 
                '10959', '2727', '10948', '10995', '10819', '10812', '3210', '10946', '3211', '10396', '2714', 
                '11036', '10421', '3201', '1525', '10397', '10395', '1528', '4510', '4508', '10889', '10888', '1526', 
                '1527', '1529', '1530', '1531', '1532', '4784', '3762', '1534', '3763', '2712', '1533', '5021', 
                '10423', '10460', '10424', '3648', '3690', '10270', '1535', '1536', '1537', '10430', '10461', 
                '10817', '582', '580', '3764', '1538', '11096', '10810', '10435', '10462', '3204', '1539', '10464', 
                '10820', '3212', '10392', '3643', '10465', '3666', '10417', '3461', '5019', '3196', '4023', '10452', 
                '5068', '1540', '5069', '5070', '1541', '5071', '10411', '2724', '1542', '10824', '2716', '10457', 
                '3191', '10420', '10412', '3032', '1543', '1544', '1545', '1546', '5199', '5201', '5200', '3761', 
                '1547', '1548', '1549', '1550', '1551', '1552', '10427', '10429', '10467', '10456', '1554', '1553', 
                '10425', '10436', '3766', '10823', '10408', '10409', '1555', '10413', '3448', '3447', '10414', '1556',
                '11045', '10816', '4509', '633', '634', '3449', '10813', '3646', '3033', '10394', '3208', '10826', 
                '1558', '1559', '3076', '3077', '10444', '4690', '1561', '1562', '10794', '10422', '10973', '10455', 
                '3767', '3771', '3770', '3768', '3769', '1560', '3772', '10821', '10822', '1563', '3036', '1564', 
                '3951', '3041', '1566', '1567', '1568', '5020', '1565', '2723', '10815', '10468', '11029', '1570', 
                '2709', '2711', '2708', '1569', '10273', '3209', '1571', '10387', '3773']

In [3]:
gis = GIS(username=user)
gis

Enter password: ········


In [4]:
lyr_reach_line = ReachFeatureLayer(url_reach_line, gis)
lyr_reach_centroid = ReachFeatureLayer(url_reach_centroid, gis)
lyr_reach_points = ReachPointFeatureLayer(url_reach_points, gis)

In [13]:
lyr_fs = lyr_reach_centroid.query(out_fields='reach_id', return_geometry=False, return_distinct_values=True)
lst_existing_reach_id = list(lyr_fs.df['reach_id'].values)

print('reaches published \t', len(lst_existing_reach_id))
print('total reaches \t\t', len(lst_reach_id_or))
lst_upload_reach_id = [reach_id for reach_id in lst_reach_id_or if reach_id not in lst_existing_reach_id]
print('reaches to publish \t', len(lst_upload_reach_id))

reaches published 	 330
total reaches 		 330
reaches to publish 	 5


In [14]:
for reach_id in lst_upload_reach_id:
    print('reach_id', reach_id)
    reach = Reach.get_from_aw(reach_id)
    print('retrieved', reach.reach_id, '-', reach.river_name, reach.reach_name)
    reach.update_putin_takeout_and_trace()
    print('\ttraced', reach.reach_id)
    reach.publish(gis, lyr_reach_line, lyr_reach_centroid, lyr_reach_points)
    print('\tpublished', reach.reach_id)

reach_id 11004
retrieved 11004 - Cow Creek, WF None
	traced 11004
	published 11004
reach_id 10995
retrieved 10995 - Little Luckiamute None
	traced 10995
	published 10995
reach_id 10461
retrieved 10461 - Oneonta Creek None
	traced 10461
	published 10461
reach_id 10816
retrieved 10816 - Smith, NF (Umpqua) None
	traced 10816
	published 10816
reach_id 3208
retrieved 3208 - Swift Creek None
	traced 3208
Parameters not valid for edit_features
	published 3208


In [11]:
reach = Reach.get_from_aw(3205)

In [13]:
reach._reach_json

{'info': {'id': 3205,
  'river': 'Clackamas, Oak Grove Fork',
  'section': 'Lower Five Miles to Clackamas River',
  'altname': None,
  'county': 'Clackamas',
  'zipcode': '260',
  'length': '5',
  'avggradient': 132,
  'maxgradient': 160,
  'agency': None,
  'gaugeinfo': None,
  'description': '<p>\n\tGradient, mile-by-mile: 160, 140, 160, 100, 100. See the description of a flow study, conducted 10/24/01, on Jason Rackley&#39;s excellent <a href="http://www.oregonkayaking.net/rivers/oakgrove/\noakgrove.html">Oregon Kayaking</a> site.</p>\n<p>\n\t&nbsp;</p>\n<p>\n\t&nbsp;</p>\n',
  'photoid': 0,
  'permitid': None,
  'huc': '17090011',
  'plat': '45.0750007629395',
  'plon': '-121.96900177002',
  'prrn': None,
  'tlat': '45.0746994018555',
  'tlon': '-122.051002502441',
  'trrn': None,
  'skid': 152640,
  'status': 'u',
  'edited': '2015-04-08 00:30:26',
  'is_final': True,
  'revision': 1204492,
  'class': 'III-V(V+)',
  'ploc': '0101000020E6100000BC749318047E5EC09A99999999894640',
  '

In [11]:
raw_json = reach._download_raw_json_from_aw()

'An easy float not far from Cottage Grove with convenient access. There are a\nnumber of fun bedrock rapids, and a couple trees to watch out for. The riparian area is thick in tree diversity, and these trees lean out over\nthe stream creating an intimate scene. For a shorter run, use one of the roadside pull-outs just upstream of Martin\nCreek. To gauge flows, go to this site:\n<http://www.nwp.usace.army.mil/Missions/Water.aspx> At the bottom is "flood and risk management". This will take you to a map of\nWillamette Valley gauges. The gauge for this creek is the Coast Fork\nWillamette near London Springs, the furthest south gauge on the map. <http://www.nwp.usace.army.mil/Water.aspx> 3/23/18 was a nice, friendly medium flow. The stream could handle a lot more\nwater.'